In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('second_cleaning_df.csv')

C:\Users\Vero\AppData\Local\Temp\ipykernel_14696\2185817358.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('second_cleaning_df.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336177 entries, 0 to 336176
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Breed                            336177 non-null  object 
 1   lifetime_cost                    336177 non-null  float64
 2   longevity                        336177 non-null  float64
 3   food_cost_year                   336177 non-null  object 
 4   grooming_required                336177 non-null  int64  
 5   size_category                    336177 non-null  object 
 6   weight_kg                        336177 non-null  float64
 7   shoulder_height_cm               336177 non-null  float64
 8   intelligence_category            336177 non-null  object 
 9   avg_food_per_week_£              336177 non-null  float64
 10  food_per_week_$                  336177 non-null  float64
 11  genetic_ailments                 336177 non-null  object 
 12  ca

In [4]:
osteo = pd.read_csv('osteo.csv')

In [5]:
osteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29734 entries, 0 to 29733
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Breed   29734 non-null  object 
 1   Gender  29734 non-null  object 
 2   Age     29328 non-null  float64
dtypes: float64(1), object(2)
memory usage: 697.0+ KB


In [6]:
import pandas as pd

# Adjust the sampling size as per the dataset size
osteo_sampled = osteo.groupby('Breed').apply(lambda x: x.sample(frac=1.0 if len(x) < 5 else 5/len(x))).reset_index(drop=True)

# Perform a left join to keep all rows from df
merged_df = pd.merge(df, osteo_sampled, how='left', on='Breed')

# Identify breeds in osteo_sampled that are not in df
extra_breeds = osteo_sampled[~osteo_sampled['Breed'].isin(df['Breed'])]

# Append the extra breeds to the merged dataframe
final_df = pd.concat([merged_df, extra_breeds], ignore_index=True)

# Check the result
print(final_df.head())
print(f"Original size of df: {len(df)}, Size after merge: {len(final_df)}")

           Breed  lifetime_cost  longevity food_cost_year  grooming_required  \
0  Border collie         20.143       12.0          324.0                2.0   
1  Border collie         20.143       12.0          324.0                2.0   
2  Border collie         20.143       12.0          324.0                2.0   
3  Border collie         20.143       12.0          324.0                2.0   
4  Border collie         20.143       12.0          324.0                2.0   

  size_category  weight_kg  shoulder_height_cm intelligence_category  \
0        medium  13.444187                51.0             Brightest   
1        medium  13.444187                51.0             Brightest   
2        medium  13.444187                51.0             Brightest   
3        medium  13.444187                51.0             Brightest   
4        medium  13.444187                51.0             Brightest   

   avg_food_per_week_£  ...  potential_for_mouthiness prey_drive  \
0                 

In [7]:
columns_to_drop = ['Gender', 'Gender_x']
final_df.drop(columns=columns_to_drop, inplace=True)

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341858 entries, 0 to 1341857
Data columns (total 30 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Breed                            1341858 non-null  object 
 1   lifetime_cost                    1341771 non-null  float64
 2   longevity                        1341771 non-null  float64
 3   food_cost_year                   1341771 non-null  object 
 4   grooming_required                1341771 non-null  float64
 5   size_category                    1341771 non-null  object 
 6   weight_kg                        1341771 non-null  float64
 7   shoulder_height_cm               1341771 non-null  float64
 8   intelligence_category            1341771 non-null  object 
 9   avg_food_per_week_£              1341771 non-null  float64
 10  food_per_week_$                  1341771 non-null  float64
 11  genetic_ailments                 1341771 non-null 

In [9]:
final_df['food_cost_year'].unique()

array([324.0, 466.0, 674.0, 405.0, 971.0, 710.0, 270.0, 1.349, 1.217,
       701.0, 440.518802683628, '440.518802683628', '$466', nan],
      dtype=object)

In [10]:
def clean_food_cost_year(value):
    if isinstance(value, str):
        # Remove the dollar sign and commas
        value = value.replace('$', '').replace(',', '.')
        # If the value is 'no data', return NaN
        if value == 'no data':
            return pd.NA
    try:
        # Convert to float
        return float(value)
    except ValueError:
        # In case of conversion failure, return NaN
        return pd.NA

# Apply the function to the column
final_df['food_cost_year'] = final_df['food_cost_year'].apply(clean_food_cost_year)

In [11]:
# Fill numeric columns with mean
numeric_cols = ['lifetime_cost', 'food_cost_year', 'weight_kg', 'longevity', 'tolerates_being_alone', 'tolerates_cold_weather', 'tolerates_hot_weather', 'prey_drive', 'tendency_to_bark_or_howl', 'energy_level', 'Age']
for col in numeric_cols:
    final_df[col].fillna(final_df[col].mean(), inplace=True)

In [12]:
# Fill numeric columns with mean
numeric_cols = ['shoulder_height_cm', 'avg_food_per_week_£', 'food_per_week_$', 'exercise_needs', 'sensitivity_level', 'incredibly_kifriendly_dogs', 'dog_friendly', 'friendly_towarstrangers', 'potential_for_mouthiness', 'wanderlust_potential', 'exercise_needs']
for col in numeric_cols:
    final_df[col].fillna(final_df[col].mean(), inplace=True)

In [13]:
# Fill integer columns with mode
integer_cols = ['grooming_required', 'adapts_well_to_apartment_living', 'Gender_y']

for col in integer_cols:
    final_df[col].fillna(final_df[col].mode()[0], inplace=True)

In [14]:
# fill object columns with mode
object_cols = ['genetic_ailments', 'Specie', 'size_category', 'intelligence_category', 'category']

for col in object_cols:
    final_df[col].fillna(final_df[col].mode()[0], inplace=True)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341858 entries, 0 to 1341857
Data columns (total 30 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Breed                            1341858 non-null  object 
 1   lifetime_cost                    1341858 non-null  float64
 2   longevity                        1341858 non-null  float64
 3   food_cost_year                   1341858 non-null  float64
 4   grooming_required                1341858 non-null  float64
 5   size_category                    1341858 non-null  object 
 6   weight_kg                        1341858 non-null  float64
 7   shoulder_height_cm               1341858 non-null  float64
 8   intelligence_category            1341858 non-null  object 
 9   avg_food_per_week_£              1341858 non-null  float64
 10  food_per_week_$                  1341858 non-null  float64
 11  genetic_ailments                 1341858 non-null 

In [16]:
# Save second_cleaning_df to a CSV file
#final_df.to_csv('dogs_expenses_total.csv', index=False)
#final_df.to_csv('C:/Users/Vero/Desktop/dogs_expenses_total.csv', index=False)

In [16]:
#keep 60% of each breed
final_df_sampled = final_df.groupby('Breed').apply(lambda x: x.sample(frac=0.6, random_state = 42)).reset_index(drop=True)

# Check the result
print(final_df_sampled.head())
print(f"Original size: {len(final_df)}, Sampled size: {len(final_df_sampled)}")

           Breed  lifetime_cost  longevity  food_cost_year  grooming_required  \
0  Affenpinscher         18.333       11.0           324.0                2.0   
1  Affenpinscher         18.333       11.0           324.0                2.0   
2  Affenpinscher         18.333       11.0           324.0                2.0   
3  Affenpinscher         18.333       11.0           324.0                2.0   
4  Affenpinscher         18.333       11.0           324.0                2.0   

  size_category  weight_kg  shoulder_height_cm intelligence_category  \
0         small  13.444187                26.0         Above average   
1         small  13.444187                26.0         Above average   
2         small  13.444187                26.0         Above average   
3         small  13.444187                26.0         Above average   
4         small  13.444187                26.0         Above average   

   avg_food_per_week_£  ...  friendly_towarstrangers potential_for_mouthiness  \

In [17]:
# Save second_cleaning_df to a CSV file
#final_df_sampled.to_csv('dogs_expenses.csv', index=False)
#final_df_sampled.to_csv('C:/Users/Vero/Desktop/dogs_expenses.csv', index=False)